# Suricata ruleset analysis

This notebook provides some visualizations on a Suricata ruleset

## Prepare the data

Prepare ruleset

In [ ]:
!/home/jovyan/.local/bin/suricata-update

Call suricata engine analysis.

In [ ]:
import shutil

In [ ]:
SURICATA_BIN = shutil.which("suricata")

In [ ]:
if SURICATA_BIN is None:
    print("suricata not in path, please provide path")
    SURICATA_BIN = input()

In [ ]:
import subprocess

In [ ]:
RESULT = subprocess.Popen([SURICATA_BIN, '-S', '/var/lib/suricata/rules/suricata.rules', '--engine-analysis', '-v'], stdout=subprocess.PIPE).communicate()

In [ ]:
print(RESULT)


Load `rules.json` from ruleset analysis

In [ ]:
import json

rules_json = open('rules.json', 'r')
ruleset = []
for line in rules_json:
    ruleset.append(json.loads(line))

In [ ]:
print(json.dumps(ruleset[0], indent=2))

Generate a simple structure with signature and mpm information

In [ ]:
ruleset_mpm =[]
for sig in ruleset:
    if 'mpm' in sig:
        ruleset_mpm.append({'raw': sig['raw'], 'id': sig['id'], 'msg': sig['msg'], 'proto': sig.get('app_proto', 'raw'), 'buffer': sig['mpm']['buffer'], 'pattern': sig['mpm']['pattern']})

## Find overused patterns

With a lot of signatures attached triggering potential long linear evaluation

In [ ]:
import networkx as nx
import html

In [ ]:
G = nx.Graph()

In [ ]:
for sig in ruleset_mpm:
    #G.add_node(sig['proto'], type='proto')
    G.add_node(sig['id'], type='sig')
    G.add_node(sig['buffer'], type='buffer')
    pattern = html.escape(sig['pattern'])
    G.add_node(pattern, type='pattern')
    G.add_edge(sig['id'], pattern)
    G.add_edge(pattern, sig['buffer'])
    #G.add_edge(sig['buffer'], sig['proto'])
    #G.add_edge(sig['buffer'], sig['id'])

In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.html.use_mathjax', False)

In [ ]:
high_degree_patterns = []
pd.set_option("max_colwidth", 440)
for node in G:
    if G.nodes[node]['type'] == 'pattern':
        if G.degree(node) > 20:
            high_degree_patterns.append([html.unescape(node), G.degree(node) - 1])
pd.DataFrame(high_degree_patterns, columns=['pattern', 'degree']).sort_values('degree', ascending=False)

In [ ]:
#pattern="|5C|pipe|5C 00|"
#pattern="&amp;cvv="
pattern="|22|method|22 3A|"
ruleset_pd = pd.DataFrame(ruleset_mpm)
pd.set_option("max_colwidth", 440)
ruleset_pd[ruleset_pd.pattern==pattern][["id", "proto", "buffer", "raw"]]

In [ ]:
import hvplot.networkx as hvnx
colors = []
for node in G:
    if G.nodes[node]['type'] == 'proto':
        colors.append('blue')
    elif G.nodes[node]['type'] == 'sig':
        colors.append('green')
    elif G.nodes[node]['type'] == 'pattern':
        colors.append('black')
    else:
        colors.append('red')

In [ ]:
DRAW_GRAPH = False

In [ ]:
if DRAW_GRAPH is True:
    pos = nx.spring_layout(G)
    hvnx.draw(G, pos, node_color=colors, node_size=40, width=800, height=400)